# 逻辑回归实现

#### 导入python库

In [17]:
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
import numpy as np

#### 封装的一些函数

In [18]:
def calc_accuracy(y_hat, y):
    # 将概率转换为类别标签: 将大于等于 0.5 的值转换为 1，小于 0.5 的值转换为 0
    y_pred = (y_hat > 0.5).astype(int) 
    correct_pred_num = np.sum(y_pred == y)    # [False,True,...,False] -> [0,1,...,0]
    accuracy = correct_pred_num / len(y)
    return accuracy

def model_train(X, y, theta, learn_step, max_epoch, loss_threshold): 
    # 通过梯度下降法循环反复迭代不断调整模型参数
    loss_value = 0
    for i in range(max_epoch):

        # 使用当前参数向量, 计算所有样本是正类的预测概率:
        z = np.dot(X, theta.T)
        y_hat = 1 / (1 + np.exp(-z))

        last_loss_value = loss_value

        # 将预测概率带入损失函数，计算损失值
        np.clip(y_hat, 1e-15, 1 - 1e-15)   #避免出现log(0)、log(1)情况
        loss_value = -1 * np.mean( y * np.log(y_hat) + ( 1 - y ) * np.log( 1 - y_hat ) )

        # 输出准确率
        if i % 100 == 0:
            accuracy = calc_accuracy(y_hat, y)
            print(f"epoch: {i}, loss: {loss_value}, acc: {accuracy}")


        # 如果损失函数已收敛，则结束循环；
        if abs(last_loss_value - loss_value) < loss_threshold:
            break
        
        # 计算损失函数 J(θ)在当前参数 θ处的梯度(变化率)
        n = X.shape[0]
        gradient = np.dot((y_hat - y), X) / n

        # 根据当前梯度值调整模型参数
        theta = theta - learn_step * gradient

    return theta


def model_verify(X, y, theta):
    # 使用模型参数，计算测试集的预测概率
    z = np.dot(X, theta.T)
    y_hat = 1 / (1 + np.exp(-z))

    # 计算准确率
    accuracy = calc_accuracy(y_hat, y)
    print("Validation Accuracy:", accuracy)


#### 参数估计

In [19]:
# 参数设置
sample_num = 150
feature_num = 10
test_sample_size = 0.3
max_epoch = 300000
loss_threshold = 1e-15
learn_step = 0.01   # 一般取值1e-2 ~ 1e-3


# 准备数据集, 并在数据集的前面增加1列全1列
X, y = make_classification(sample_num, feature_num) 
column_ones = np.ones((X.shape[0], 1))          # 参数是一个元组，表示行和列
X = np.concatenate((column_ones, X), axis=1)        # axis=1水平方向连接， axis=0 竖直...

# 拆分数据集: 训练数据集、测试数据集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_sample_size)


# 初始化模型参数, 注意要包含偏置项bias
theta = np.random.randn(1, feature_num + 1) 
theta = np.zeros(feature_num + 1)

# 训练模型
theta = model_train(X_train, y_train, theta, learn_step, max_epoch, loss_threshold)

# 验证模型
model_verify(X_test, y_test, theta)

# 保存模型参数
np.save('theta.npy', theta)

epoch: 0, loss: 0.6931471805599453, acc: 0.5142857142857142
epoch: 100, loss: 0.546908967977251, acc: 0.8285714285714286
epoch: 200, loss: 0.4772295127183046, acc: 0.8285714285714286
epoch: 300, loss: 0.4383927799990948, acc: 0.8380952380952381
epoch: 400, loss: 0.4143380002648339, acc: 0.8380952380952381
epoch: 500, loss: 0.3982892426824595, acc: 0.8380952380952381
epoch: 600, loss: 0.38697458866304363, acc: 0.8380952380952381
epoch: 700, loss: 0.37865049058872124, acc: 0.8380952380952381
epoch: 800, loss: 0.37231545268151184, acc: 0.8476190476190476
epoch: 900, loss: 0.3673593201615358, acc: 0.8476190476190476
epoch: 1000, loss: 0.36339230850334575, acc: 0.8476190476190476
epoch: 1100, loss: 0.3601554948664514, acc: 0.8476190476190476
epoch: 1200, loss: 0.35747114608086344, acc: 0.8476190476190476
epoch: 1300, loss: 0.355213787632617, acc: 0.8476190476190476
epoch: 1400, loss: 0.3532926527172253, acc: 0.8476190476190476
epoch: 1500, loss: 0.3516406586894226, acc: 0.8476190476190476
e

#### 模型预测

In [20]:
# 从文件中加载模型参数
theta = np.load('theta.npy')
print(theta)

# 准备数据集, 并在数据集的前面增加1列全1列
feature_num = 10       # 样本的数量
sample_num = 150       # 样本特征的个数
X, y = make_classification(sample_num, feature_num) 
column_ones = np.ones((X.shape[0], 1))          # 参数是一个元组，表示行和列
X = np.concatenate((column_ones, X), axis=1)        # axis=1水平方向连接， axis=0 竖直...

# 使用模型参数，计算所有样本是正类的预测概率
z = np.dot(X, theta.T)
y_hat = 1 / (1 + np.exp(-z))

# 计算准确率
accuracy = calc_accuracy(y_hat, y)
print("Validation Accuracy:", accuracy)

[ 0.49516105  2.45709664 -0.27988594 -0.36977526  0.1542271  -0.1644425
 -0.23786215  0.23505187 -0.43441541  0.13597378 -0.0729565 ]
Validation Accuracy: 0.4266666666666667
